<table style="margin: auto; background-color: white;">
    <tr>
      <td style="background-color: white;">
        <img src='https://drive.google.com/uc?export=view&id=1lgflViz1uefcvVW1iI57haB4M1bKsZtp' alt="drawing" width="200" />
      </td>
      <td style="background-color: white;">
        <img src='https://drive.google.com/uc?export=view&id=1R6PphT9jmd2vikODFPf6cW54QtZ29o2a' alt="drawing" width="200" />
      </td>
      <td style="background-color: white;">
        <img src='https://drive.google.com/uc?export=view&id=1lgflViz1uefcvVW1iI57haB4M1bKsZtp' alt="drawing" width="200" />
      </td>
      <td style="background-color: white;">
        <img src='https://drive.google.com/uc?export=view&id=1R6PphT9jmd2vikODFPf6cW54QtZ29o2a' alt="drawing" width="200" />
      </td>
      <td style="background-color: white;">
        <img src='https://drive.google.com/uc?export=view&id=1lgflViz1uefcvVW1iI57haB4M1bKsZtp' alt="drawing" width="200" />
      </td>
      <td style="background-color: white;">
        <img src='https://drive.google.com/uc?export=view&id=1R6PphT9jmd2vikODFPf6cW54QtZ29o2a' alt="drawing" width="200" />
      </td>
      <td style="background-color: white;">
        <img src='https://drive.google.com/uc?export=view&id=1lgflViz1uefcvVW1iI57haB4M1bKsZtp' alt="drawing" width="200" />
      </td>
    </tr>
</table>

#TUTORIAL 3

Bem-vindo! Neste tutorial você aprenderá sobre a interface de programação da plataforma **Flautim** e também como montar um experimento simples de classificação usando o dataset [MNIST](https://github.com/zalandoresearch/fashion-mnist).

O código desse tutorial pode ser acessado em: [clique aqui](https://github.com/FutureLab-DCC/flautim_tutoriais/tree/main/TUTORIAL_3).


Vamos começar entendendo a interface de programação da **Flautim**, representada na figura abaixo. A **Flautim_api** é uma biblioteca modularizada que facilita a realização de experimentos de aprendizado de máquina, seja convencional/centralizado ou federado.

Todo projeto **Flautim** precisa herdar essa biblioteca, que contém submódulos específicos para diferentes tecnologias (por exemplo, submódulos para PyTorch, TensorFlow, etc). Neste tutorial usaremos o submódulo para PyTorch.

<div style="text-align: center;"> <table style="margin: auto;"> <tr> <td> <img src='https://drive.google.com/uc?export=view&id=1QOI4jWrwS979xhW_wlGzkPa2bMA-giuc' alt="Interface da plataforma Flautim" width="800" /> </td> </tr> </table> </div>


Dentro de cada submódulo existem três componentes principais (classes):

**1. Dataset:** é utilizado para representar os dados do experimento. Esta classe pode ser reutilizada em diversos experimentos e com diferentes modelos, sendo o componente mais versátil e reutilizável. Os usuários podem importar os dados de diversas fontes, como arquivos locais ou bases de dados online, desde que a classe Dataset seja herdada.

**2. Model:** representa qualquer conjunto de parâmetros treináveis dentro do projeto. Ela permite a aplicação de técnicas de aprendizado de máquina por meio de treinamento desses parâmetros. No caso de PyTorch, a classe herda a nn.Module, que define a estrutura e os parâmetros treináveis do modelo.

**3. Experiment:** define o ciclo de treinamento e validação. Existem dois tipos principais de experimentos: o experimento centralizado, que segue o fluxo
convencional de aprendizado de máquina, e o experimento federado, adaptado para
aprendizado federado. Esta classe inclui duas funções principais, um loop de
treinamento e um loop de validação, que realizam a atualização dos parâmetros e
cálculo das métricas de custo, respectivamente.

Além desses três componentes principais, há também um módulo chamado Common. Este módulo fornece acesso a classes essenciais para o gerenciamento de dados e monitoramento do treinamento.


Com essa visão geral, você está pronto para começar montar seus próprios experimentos. Vamos ao passo a passo!

###Passo 1: Criando o dataset que será usado no experimento

Um conjunto de dados no Flautim é acessado por um arquivo .py que deve conter uma classe que herda de Dataset.

**Exemplo: Implementando a Classe MNISTDataset**

O código abaixo implementa uma classe MNISTDataset utilizando o dataset MNIST para resolver um problema de classificação.

In [ ]:
from flautim.pytorch.Dataset import Dataset
import numpy as np
import pandas as pd
import torch
import copy

from torch.utils.data import DataLoader

class MNISTDataset(Dataset):

    def __init__(self, FM_Normalization, EVAL_Transforms, TRAIN_Transforms, partition, **kwargs):

        name = kwargs.get('name', 'MNIST')

        super(MNISTDataset, self).__init__(name, **kwargs)

        self.FM_Normalization = FM_Normalization
        self.EVAL_Transforms = EVAL_Transforms
        self.TRAIN_Transforms = TRAIN_Transforms

        self.feature_name = kwargs.get("feature_name", 'image')
        self.split = kwargs.get("split_data", True)

        if self.split:
            partition = partition.train_test_split(test_size=0.2, seed=42)

        self.train_partition = partition["train"]
        self.test_partition = partition["test"]

    def dataloader(self, validation = False):
        tmp = self.validation() if validation else self.train()
        return DataLoader(tmp, batch_size = self.batch_size, num_workers = 1)



    def apply_train_transforms(self, batch):
        """Apply transforms to the partition from FederatedDataset."""
        batch[self.feature_name] = [self.TRAIN_Transforms(img) for img in batch[self.feature_name]]
        return batch


    def  apply_eval_transforms(self, batch):
        """Apply transforms to the partition from FederatedDataset."""
        batch[self.feature_name] = [self.EVAL_Transforms(img) for img in batch[self.feature_name]]
        return batch


    def train(self):

        return self.train_partition.with_transform(self.apply_train_transforms)


    def validation(self):

        return self.test_partition.with_transform(self.apply_eval_transforms)

###Passo 2: Criando o modelo que será usado no experimento

Agora, vamos criar a classe que implementa o modelo. Essa classe deve herdar da classe Model.


**Exemplo: Implementando a Classe MNISTModel**

A classe Net implementa uma rede neural convolucional baseada na rede LeNet, com as seguintes camadas:
* Uma camada convolucional com entrada de 1 canal, saída de 6 canais e kernel de tamanho 5×5;
* Uma camada de Max Pooling com janela 2 e stride 2;
* Uma segunda camada convolucional com entrada de 6 canais, saída de 16 canais e kernel de tamanho 5×5;
* Uma camada totalmente conectada com 120 neurônios;
* Uma camada totalmente conectada com 84 neurônios;
* Uma camada de saída totalmente conectada com número de neurônios igual ao número de classes do problema

Essa classe pode ser incluída, por exemplo, em um arquivo MnistModel.py

In [ ]:
from flautim.pytorch.Model import Model
import torch

import torch.nn as nn
import torch.nn.functional as F


class MNISTModel(Model):
    def __init__(self, context, num_classes: int, **kwargs) -> None:
        super(MNISTModel, self).__init__(context, name = "MNIST", version = 1, id = 1, **kwargs)
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 4 * 4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, num_classes)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 4 * 4)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

###Passo 3: Criando o experimento

Por fim, será criado o experimento, isto é, uma classe que implementa os loops de treinamento e validação do modelo MNISTModel no dataset MNISTDataset. Para isso, precisamos criar dois arquivos .py, o run.py (que deve ter obrigatoriamente esse nome) e o .py responsável por implementar o experimento, descritos a seguir:

**1. Arquivo run.py:**

* Esse arquivo é o ponto de entrada de todo experimento Flautim, pois é ele
que deve iniciar a classe do experimento e também um modelo e um Dataset.

**2. Arquivo .py do experimento:**

* Esse arquivo deve conter uma classe que implemente os métodos de treinamento (training_loop) e validação (evaluation_loop) do modelo. Essa classe deve herdar da classe Experiment.

Esse tutorial cobrirá dois tipos de experimentos, um experimento centralizado e outro descentralizado. Portanto, o passo 3 será dividido entre esses dois cenários.

####Passo 3.1: Experimento centralizado

**Implementando a Classe MNISTExperiment**

No código abaixo, a classe MNISTExperiment foi criada no modo centralizado com seus métodos training_loop e evaluation_loop para treinar e testar a rede neural. Esses métodos retornam o valor da função de perda e a acurácia de treinamento e de validação.

In [ ]:
from flautim.pytorch.centralized.Experiment import Experiment

import flautim as fl
import flautim.metrics as flm
import numpy as np
import torch
import time

class MNISTExperiment(Experiment):
    def __init__(self, model, dataset, context, **kwargs):
        super(MNISTExperiment, self).__init__(model, dataset, context, **kwargs)

        self.criterion = torch.nn.CrossEntropyLoss()
        self.lr = kwargs.get('lr', 0.001)
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=self.lr)
        self.epochs = kwargs.get('epochs', 30)
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


    def training_loop(self, data_loader):

        self.model.to(self.device)
        self.model.train()

        running_loss = 0.0

        yhat, y_real = [], []

        for batch in data_loader:
            images = batch["image"]
            labels = batch["label"]
            self.optimizer.zero_grad()

            outputs = self.model(images.to(self.device))
            loss = self.criterion(outputs, labels.to(self.device))
            loss.backward()
            self.optimizer.step()

            running_loss += loss.item()

            _, predicted = torch.max(outputs.data, 1)
            yhat.append(predicted.detach().cpu())
            y_real.append(labels.detach().cpu())

        accuracy = self.metrics.ACCURACY(torch.cat(yhat).numpy(), torch.cat(y_real).numpy())
        accuracy_2 = self.metrics.ACCURACY_2(torch.cat(yhat).numpy(), torch.cat(y_real).numpy())
        avg_trainloss = running_loss / len(data_loader)
        return float(avg_trainloss), {'ACCURACY': accuracy, 'ACCURACY_2': accuracy_2}

    def validation_loop(self, data_loader):

        self.model.to(self.device)
        self.model.eval()

        yhat, y_real = [], []

        loss = 0.0
        with torch.no_grad():
            for batch in data_loader:
                images = batch["image"].to(self.device)
                labels = batch["label"].to(self.device)
                outputs = self.model(images)
                loss += self.criterion(outputs, labels).item()
                _, predicted = torch.max(outputs.data, 1)
                yhat.append(predicted.detach().cpu())
                y_real.append(labels.detach().cpu())

        accuracy = self.metrics.ACCURACY(torch.cat(yhat).numpy(), torch.cat(y_real).numpy())
        accuracy_2 = self.metrics.ACCURACY_2(torch.cat(yhat).numpy(), torch.cat(y_real).numpy())
        loss = loss / len(data_loader)

        return float(loss), {'ACCURACY': accuracy, 'ACCURACY_2': accuracy_2}

**Implementando o run.py para realização de um experimento centralizado**

**1. Upload do Conjunto de Dados:**

* *Arquivo Local:* Se o seu conjunto de dados for um arquivo (por exemplo, CSV, NPZ, etc.), faça o upload para a plataforma e carregue-o usando o caminho ./data/nomedoarquivo.

* *URL:* Se o conjunto de dados estiver disponível em uma URL, inclua a URL no seu código e carregue-o diretamente.

**2. Crie uma instância para MNISTDataset, MNISTModel, MNISTExperiment.**

**3. Execute as funções:**

* ***experiment.run:*** Executa o experimento centralizado.

In [ ]:
import flautim as fl
import flautim.metrics as flm
import MNISTDataset, MNISTModel, MNISTExperiment

import torch
import numpy as np
import pandas as pd

from datasets import load_dataset

from torchvision.transforms import (
    Compose,
    Normalize,
    RandomCrop,
    RandomHorizontalFlip,
    ToTensor,
)


FM_NORMALIZATION = ((0.1307,), (0.3081,))
EVAL_TRANSFORMS = Compose([ToTensor(), Normalize(*FM_NORMALIZATION)])
TRAIN_TRANSFORMS = Compose(
    [
        RandomCrop(28, padding=4),
        RandomHorizontalFlip(),
        ToTensor(),
        Normalize(*FM_NORMALIZATION),
    ]
)

DATASET="ylecun/mnist"

if __name__ == '__main__':

    context = fl.init()

    fl.log(f"Flautim inicializado!!!")

    partition = load_dataset("zalando-datasets/fashion_mnist")

    dataset = MNISTDataset.MNISTDataset(FM_NORMALIZATION, EVAL_TRANSFORMS, TRAIN_TRANSFORMS, partition, split_data=False, batch_size = 32, shuffle = False, num_workers = 0)

    model = MNISTModel.MNISTModel(context, num_classes = 10)

    fl.log(f"Modelo criado!!!")

    experiment = MNISTExperiment.MNISTExperiment(model, dataset,  context)

    fl.log(f"Experimento criado!!!")

    # Exemplo de métrica implementada pelo usuário
    def accuracy_2(y, y_hat):
        y = np.asarray(y)
        y_hat = np.asarray(y_hat)
        return np.mean(y == y_hat)

    # Adiciona a métrica ao módulo de métricas
    flm.Metrics.accuracy_2 = accuracy_2

    fl.log(f"Métrica adicionada!!!")

    experiment.run(metrics = {'ACCURACY': flm.Metrics.accuracy, 'ACCURACY_2': flm.Metrics.accuracy_2})

####Passo 3.2: Experimento federado
**Implementando a Classe MNISTExperiment**

No código abaixo, criamos a classe MNISTExperiment no modo federado com seus métodos training_loop e evaluation_loop para treinar e testar a rede neural. Esses métodos retornam o valor da função de perda e a acurácia de treinamento e de validação.

In [ ]:
from flautim.pytorch.federated.Experiment import Experiment
import flautim as fl
import flautim.metrics as flm
import numpy as np
import torch
import time

class MNISTExperiment(Experiment):
    def __init__(self, model, dataset, context, **kwargs):
        super(MNISTExperiment, self).__init__(model, dataset, context, **kwargs)

        self.criterion = torch.nn.CrossEntropyLoss()
        self.lr = kwargs.get('lr', 0.01)
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=self.lr)
        self.epochs = kwargs.get('epochs', 30)
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    # Exemplo de métrica implementada pelo usuário
    def accuracy_2(y, y_hat):
        y = np.asarray(y)
        y_hat = np.asarray(y_hat)
        return np.mean(y == y_hat)

    # Adiciona a métrica ao módulo de métricas
    flm.Metrics.accuracy_2 = accuracy_2

    def training_loop(self, data_loader):

        self.model.to(self.device)
        self.model.train()

        correct, running_loss = 0.0, 0.0

        yhat, y_real = [], []

        for batch in data_loader:
            images = batch["image"]
            labels = batch["label"]
            self.optimizer.zero_grad()

            outputs = self.model(images.to(self.device))
            loss = self.criterion(outputs, labels.to(self.device))
            loss.backward()
            self.optimizer.step()

            running_loss += loss.item()
            correct += (torch.max(outputs, 1)[1].cpu() == labels.cpu()).sum().item()

            _, predicted = torch.max(outputs.data, 1)
            yhat.append(predicted.detach().cpu())
            y_real.append(labels.detach().cpu())

        accuracy = flm.Metrics.accuracy(torch.cat(yhat).numpy(), torch.cat(y_real).numpy())
        accuracy_2 = flm.Metrics.accuracy_2(torch.cat(yhat).numpy(), torch.cat(y_real).numpy())
        avg_trainloss = running_loss / len(data_loader)
        return float(avg_trainloss), {'ACCURACY': accuracy, "ACCURACY_2": accuracy_2}

    def validation_loop(self, data_loader):

        self.model.to(self.device)
        self.model.eval()

        correct, loss = 0, 0.0

        yhat, y_real = [], []

        with torch.no_grad():
            for batch in data_loader:
                images = batch["image"].to(self.device)
                labels = batch["label"].to(self.device)
                outputs = self.model(images)
                loss += self.criterion(outputs, labels).item()
                correct += (torch.max(outputs, 1)[1].cpu() == labels.cpu()).sum().item()

                _, predicted = torch.max(outputs.data, 1)
                yhat.append(predicted.detach().cpu())
                y_real.append(labels.detach().cpu())

        accuracy = flm.Metrics.accuracy(torch.cat(yhat).numpy(), torch.cat(y_real).numpy())
        accuracy_2 = flm.Metrics.accuracy_2(torch.cat(yhat).numpy(), torch.cat(y_real).numpy())
        loss = loss / len(data_loader)

        return float(loss), {'ACCURACY': accuracy, "ACCURACY_2": accuracy_2}

**Implementando o run.py para realização de um experimento federado**

**1. Upload do Conjunto de Dados:**

* *Arquivo Local:* Se o seu conjunto de dados for um arquivo (por exemplo, CSV, NPZ, etc.), faça o upload para a plataforma e carregue-o usando o caminho ./data/nomedoarquivo.

* *URL:* Se o conjunto de dados estiver disponível em uma URL, inclua a URL no seu código e carregue-o diretamente.

**2. Separação dos dados por cliente:**

* Para simular 4 clientes, divida os dados em 4 partes.

**3. Crie uma instância para MNISTDataset, MNISTModel, MnistExperiment.**

**4. Execute as funções:**
* ***generate_server_fn:*** Cria a estratégia para o aprendizado federado
* ***generate_client_fn:*** Gera o modelo e o dataset de cada cliente.
* ***evaluate_fn:*** Avalia o modelo global usando o dataset de um dos clientes.
* ***run_federated:*** Executa o experimento federado.

In [ ]:
from flautim.pytorch.common import run_federated, weighted_average
from flautim.pytorch import Model, Dataset
from flautim.pytorch.federated import Experiment
import MNISTDataset, MNISTModel, MnistExperiment
import flautim as fl
from flwr_datasets import FederatedDataset
from flwr_datasets.partitioner import DirichletPartitioner
from flwr.common import Context, ndarrays_to_parameters
import flwr
import pandas as pd
import numpy as np
from flwr.server import ServerConfig, ServerAppComponents
from datasets import load_dataset

from torchvision.transforms import (
    Compose,
    Normalize,
    RandomCrop,
    RandomHorizontalFlip,
    ToTensor,
)


FM_NORMALIZATION = ((0.1307,), (0.3081,))
EVAL_TRANSFORMS = Compose([ToTensor(), Normalize(*FM_NORMALIZATION)])
TRAIN_TRANSFORMS = Compose(
    [
        RandomCrop(28, padding=4),
        RandomHorizontalFlip(),
        ToTensor(),
        Normalize(*FM_NORMALIZATION),
    ]
)

DATASET = "zalando-datasets/fashion_mnist"
NUM_PARTITIONS = 30

def fit_config(server_round: int):
    """Return training configuration dict for each round.

    Perform two rounds of training with one local epoch, increase to two local
    epochs afterwards.
    """
    config = {
        "server_round": server_round,  # The current round of federated learning
    }
    return config



def generate_server_fn(context, eval_fn, **kwargs):

    def create_server_fn(context_flwr:  Context):

        net = MNISTModel.MNISTModel(context, num_classes = 10, suffix = 0)
        params = ndarrays_to_parameters(net.get_parameters())

        strategy = flwr.server.strategy.FedAvg(
                          initial_parameters=params,
                          evaluate_metrics_aggregation_fn=weighted_average,
                          fraction_fit=0.1,  # 10% clients sampled each round to do fit()
                          fraction_evaluate=0.5,  # 50% clients sample each round to do evaluate()
                          evaluate_fn=eval_fn,
                          on_fit_config_fn = fit_config,
                          on_evaluate_config_fn = fit_config
                          )
        num_rounds = 30
        config = ServerConfig(num_rounds=num_rounds)

        return ServerAppComponents(config=config, strategy=strategy)
    return create_server_fn

def generate_client_fn(context):

    def create_client_fn(context_flwr:  Context):

        global fds

        cid = int(context_flwr.node_config["partition-id"])

        partition = fds.load_partition(cid)

        model = MNISTModel.MNISTModel(context, num_classes = 10, suffix = cid)

        dataset = MNISTDataset.MNISTDataset(FM_NORMALIZATION, EVAL_TRANSFORMS, TRAIN_TRANSFORMS, partition, batch_size = 32, shuffle = False, num_workers = 0)

        return MNISTExperiment.MNISTExperiment(model, dataset,  context).to_client()

    return create_client_fn


def evaluate_fn(context):
    def fn(server_round, parameters, config):
        """This function is executed by the strategy it will instantiate
        a model and replace its parameters with those from the global model.
        The, the model will be evaluate on the test set (recall this is the
        whole MNIST test set)."""

        global FM_NORMALIZATION, EVAL_TRANSFORMS, TRAIN_TRANSFORMS, DATASET
        global fds

        model = MNISTModel.MNISTModel(context, num_classes = 10, suffix = "FL-Global")
        model.set_parameters(parameters)

        partition = fds.load_partition(0)

        dataset = MNISTDataset.MNISTDataset(FM_NORMALIZATION, EVAL_TRANSFORMS, TRAIN_TRANSFORMS, partition, batch_size = 32, shuffle = False, num_workers = 0)
        dataset.test_partition = load_dataset(DATASET)["test"]

        experiment = MNISTExperiment.MNISTExperiment(model, dataset, context)

        config["server_round"] = server_round

        loss, _, return_dic = experiment.evaluate(parameters, config)

        return loss, return_dic

    return fn

partitioner = DirichletPartitioner(
            num_partitions=NUM_PARTITIONS,
            partition_by="label",
            alpha=1.0,
            seed=42,
        )
fds = FederatedDataset(
            dataset=DATASET,
            partitioners={"train": partitioner},
        )

fds.load_partition(0)

if __name__ == '__main__':

    context = fl.init()

    fl.log(f"Flautim inicializado!!!")


    client_fn_callback = generate_client_fn(context)
    evaluate_fn_callback = evaluate_fn(context)
    server_fn_callback = generate_server_fn(context, eval_fn = evaluate_fn_callback)

    fl.log(f"Experimento criado!!!")

    run_federated(client_fn_callback, server_fn_callback, num_clients = NUM_PARTITIONS)